In [ ]:
# load dotenv + imports for retriever tool
import os
from dotenv import load_dotenv
from langchain_chroma import Chroma 
from langchain.tools import tool
import langchainhub as hub
import chromadb
from langchain_google_genai import GoogleGenerativeAIEmbeddings

load_dotenv()

# Load environment variables from chromaDB --CloudClient
client = chromadb.CloudClient(
    api_key=os.getenv("CHROMA_API_KEY"),
    tenant = os.getenv("CHROMA_TENANT"),
    database=os.getenv("CHROMA_DATABASE")
    
)
collection = client.get_collection(name="nice_org")

# connecting embeddings with Gemini
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

vectorstore = Chroma(client=client,
                     collection_name="nice_org",
                     embedding_function=embeddings)

In [ ]:
import typing_extensions
print(typing_extensions.__version__)

In [ ]:
import sys
!{sys.executable} -m pip install --upgrade --force-reinstall typing-extensions pydantic pydantic-core
